In [ ]:
# !curl -fLo NL-shapes.zip https://www.eea.europa.eu/data-and-maps/data/eea-reference-grids-2/gis-files/netherlands-shapefile/at_download/file
!curl -fLo NL-municipalities.zip https://opendata.arcgis.com/datasets/e1f0dd70abcb4fceabbc43412e43ad4b_0.zip

In [ ]:
import zipfile

# with zipfile.ZipFile('NL-shapes.zip') as shapes_zip_file:
#     shapes_zip_file.extractall(path='nl_shapes')
with zipfile.ZipFile('NL-municipalities.zip') as shapes_zip_file:
    shapes_zip_file.extractall(path='nl_municipalities')

In [ ]:
import matplotlib.pyplot as plt
import geopandas as gp

In [ ]:
shapes = gp.read_file('nl_shapes/nl_1km.shp')

In [ ]:
shapes.head()

In [ ]:
shapes.plot()

In [ ]:
gemeentes = gp.read_file('nl_municipalities/Gemeentegrenzen_2019.shp')
gemeentes = gemeentes.to_crs("EPSG:3395")

In [ ]:
gemeentes.head()

In [ ]:
gemeentes.Code = gemeentes.Code.astype('int')

In [ ]:
gemeentes.plot(figsize=(10, 12))

# Download municpality residents data

Go to https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=37230ned&_theme=251 and select the data you need; I picked only "Bevolking aan het einde van de periode" (residents at end of period).

In [ ]:
import pandas as pd
import datetime

In [ ]:
residents = pd.read_csv('37230ned_UntypedDataSet_17032020_111452.csv', sep=';', index_col=0)

In this experiment we only care about the latest numbers, so let's filter and remove the Perioden (periods) column after that:

In [ ]:
residents_latest = residents[residents.Perioden == '2020MM01']\
                            .drop(columns='Perioden')

In [ ]:
residents_latest

Also, we only care about municipalities, so let's get rid of any non-municipal data and clean the region number column so we can easily match with the shapes data.

In [ ]:
gemeente_residents = residents_latest[residents_latest.RegioS.str.contains('GM')]

In [ ]:
gemeente_residents['Code'] = gemeente_residents.RegioS.str.strip('GM')
gemeente_residents = gemeente_residents.drop(columns='RegioS')

In [ ]:
gemeente_residents.head()

No idea why this still has NaNs... but ok, let's filter those as well (they also appear in other years, so maybe those municipalities haven't been counted recently).

In [ ]:
gemeente_residents.dropna(inplace=True)

In [ ]:
gemeente_residents.BevolkingAanHetEindeVanDePeriode_15 = gemeente_residents.BevolkingAanHetEindeVanDePeriode_15.astype('int')

In [ ]:
gemeente_residents.Code = gemeente_residents.Code.astype('int')

In [ ]:
len(gemeente_residents), len(gemeentes)

Ok, nice, equal count, let's see if we can match:

In [ ]:
gemeentes[gemeentes.Code == 1719]

In [ ]:
gemeente_residents[gemeente_residents.Code == 1719]

In [ ]:
gemeentes_data = gemeentes.merge(gemeente_residents)

In [ ]:
gemeentes_data.plot(column='BevolkingAanHetEindeVanDePeriode_15', figsize=(10, 12))

Great, that seems about right! Would normally plot logarithmically, but that's not the issue at hand right now.

# Corona data

The RIVM publishes data on Corona infections per municipality daily. I can't seem to get a URL to the dataset, so just download it via the context menu (download CSV).

In [ ]:
corona_reports = pd.read_csv('Aantal Coronavirus (COVID-19)-meldingen.csv', sep=';', header=0,
                             names=['Gemeentena', 'corona_number'],
                             dtype={'Gemeentena': 'str', 'corona_number': 'int'})

In [ ]:
corona_reports

Ugh, ok, so let's see if we can also merge on name, leaving unmentioned municipalities at NaN (assuming that's the RIVM's data format).

In [ ]:
gemeentes_data2 = gemeentes_data.merge(corona_reports, how='left', on='Gemeentena')

In [ ]:
(gemeentes_data2.corona_number == 0).sum()

In [ ]:
gemeentes_data2.corona_number[gemeentes_data2.corona_number.isna()] = 0

In [ ]:
(gemeentes_data2.corona_number == 0).sum()

Wow, no hickups at all, unreal. Ok, forward to our final goal!

# Corona density map

We plot the number of confirmed COVID-19 infections per capita per municipality.

In [ ]:
gemeentes_data2['promille'] = gemeentes_data2['corona_number'] / gemeentes_data2['BevolkingAanHetEindeVanDePeriode_15'] * 1000

In [ ]:
gemeentes_data2

In [ ]:
gemeentes_data2.plot(column='promille', figsize=(10, 12))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 12))
gemeentes_data2.plot(column='promille', scheme='quantiles',
                     ax=ax, legend=True,
                     legend_kwds={'title': "Reported Corona cases\nper 1000 inhabitants", 'loc':'upper left'})